In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd

import os
from liams_funcs import *
from segmentation import *

from cellpose import models, io

In [ ]:
control_dir = '../control_images/'
drug_dir = '../drug_images/'

control_fnames = [control_dir + fname for fname in os.listdir(control_dir)]
drug_fnames = [drug_dir + fname for fname in os.listdir(drug_dir)]

In [ ]:
# Directory paths
dir_hist = '../corrected_images/hist_correction/'
dir_adapthist = '../corrected_images/adapthist_correction/'

# Function to check if directory exists
def check_directory_exists(directory):
    try:
        if os.path.exists(directory) and os.path.isdir(directory):
            print(f"Directory exists: {directory}")
        else:
            raise NotADirectoryError(f"{directory} is not a valid directory.")
    except Exception as e:
        print(f"Error checking directory {directory}: {e}")

# Check if directories exist
check_directory_exists(dir_hist)
check_directory_exists(dir_adapthist)
check_directory_exists(drug_dir)
check_directory_exists(control_dir)

In [ ]:
diameters = np.arange(10, 200+1, 10)
model = models.Cellpose(model_type='nuclei', gpu=True)

N_segments = []
Areas = []
for diameter in tqdm(diameters):
    n_segments = []
    areas = []
    for fname in control_fnames + drug_fnames:
        image = io.imread(dir_hist + fname.split('/')[-1])

        masks = model.eval(image, channels=[1,0], diameter=diameter)[0]

        n_segments.append(masks.max())
        areas.append(np.sum(masks != 0))

    N_segments.append(n_segments)
    Areas.append(areas)

N_segments = np.asarray(N_segments)
Areas = np.asarray(Areas)

In [ ]:
fig, (l, r) = plt.subplots(1, 2, figsize=(12,4))

l.set_prop_cycle(plt.cycler('color', plt.cm.Reds(np.linspace(0.5, 1, len(control_fnames)))))
r.set_prop_cycle(plt.cycler('color', plt.cm.Blues(np.linspace(0.5, 1, len(control_fnames)))))


l.plot(diameters, N_segments, '-o')
l.plot(diameters, N_segments.mean(axis=1), c='aqua', linewidth=3)
l.set_yscale('log'); l.grid(axis='y')
l.set_xlim(diameters[0], diameters[-1])#; l.set_ylim(0)
l.set_yticks([10, 50, 100, 200, 500], [10, 50, 100, 200, 500])
l.set_xlabel('Diameter (pix)'); l.set_ylabel('# of segments')

r.plot(diameters, Areas, '-o')
r.plot(diameters, Areas.mean(axis=1), c='gold', linewidth=3)
r.set_yscale('log')
r.set_xlim(diameters[0], diameters[-1])#; r.set_ylim(0)
r.set_xlabel('Diameter (pix)'); r.set_ylabel(r'Area covered (pix$^2$)')

fig.tight_layout()
fig.savefig('../figures/diameter_analysis.png', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
fname1 = '../control_images/001_z26.png'
fname2 = '../drug_images/011_z57.png'

In [ ]:
_ = sequential_segmentation(fname1, diams=[80, 80, 80], threshold=0, show=True, save_to='../figures/control_segmentation.png')
_ = sequential_segmentation(fname2, diams=[80, 80, 80], threshold=0, show=True, save_to='../figures/drug_segmention.png')

In [ ]:
_ = combined_segmentation(fname1, cell_diams=[100, 100, 100], nucleus_diams=[80, 80, 80], show_plot=True, min_coverage=0.95, save_to='../figures/combined_segmentation_95.png')
_ = combined_segmentation(fname1, cell_diams=[100, 100, 100], nucleus_diams=[80, 80, 80], show_plot=True, min_coverage=0.97, save_to='../figures/combined_segmentation_97.png')
_ = combined_segmentation(fname1, cell_diams=[100, 100, 100], nucleus_diams=[80, 80, 80], show_plot=True, min_coverage=0.99, save_to='../figures/combined_segmentation_99.png')

In [ ]:
_ = combined_segmentation(fname1, show_plot=True, save_to='../figures/control_combined_segmentation.png')
_ = combined_segmentation(fname2, show_plot=True, save_to='../figures/drug_combined_segmentation1.png')

In [ ]:
dir = 'segments/control/'
for fname in control_fnames:
    new_fname = dir + fname.split('/')[-1] + '.npy'
    print(new_fname)
    cells, nuclei = combined_segmentation(fname)
    save_to(cells, nuclei, new_fname)

dir = 'segments/drug/'
for fname in drug_fnames:
    new_fname = dir + fname.split('/')[-1] + '.npy'
    print(new_fname)
    cells, nuclei = combined_segmentation(fname)
    save_to(cells, nuclei, new_fname)